In [1]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat

In [2]:
from tensorflow.python.client import device_lib


from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)
print(device_lib.list_local_devices())


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11950971130930078468
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17507853541075786868
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10377517648998148989
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7009469728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4405632696673838299
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capabil

In [3]:
directory = "./data/eeg_feature_smooth/1/"
directories = ["./data/eeg_feature_smooth/{}/".format(i+1) for i in range(3)] 
print(directories)


['./data/eeg_feature_smooth/1/', './data/eeg_feature_smooth/2/', './data/eeg_feature_smooth/3/']


In [4]:
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories),len(os.listdir(directories[0])), n, 4, 62, 5, 64)) # features = 4*62*5*64 (zero padded) // trials = (3 sessions) x 15 people x 24 labels 
li = []
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(dire + file) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0,2,1))
                #print(sample.shape)
                sample = np.pad(sample, [(0,0), (0,0), (0, 64-sample.shape[2])])
                #print(sample.shape)
                array[h][i][j][k] = sample

print(array.shape)



./data/eeg_feature_smooth/1/
./data/eeg_feature_smooth/2/
./data/eeg_feature_smooth/3/
(3, 15, 24, 4, 62, 5, 64)


In [5]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])

In [6]:
_X.shape

(1080, 4, 62, 5, 64)

In [7]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)
y.shape

(1080,)

In [8]:
X = _X.transpose(0, 4, 1, 2,3)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
X.shape

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 62, 5)


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(
32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 62, 32)            119072    
_________________________________________________________________
flatten (Flatten)            (None, 1984)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               254080    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 374,442
Trainable params: 374,442
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model.fit(X_train, y_train,epochs=30)

Epoch 1/30
27/27 [==============================] - 0s 11ms/step - loss: 135596096.0000 - accuracy: 0.3067
Epoch 2/30
27/27 [==============================] - 0s 10ms/step - loss: 22624178.0000 - accuracy: 0.3542
Epoch 3/30
27/27 [==============================] - 0s 10ms/step - loss: 7586141.5000 - accuracy: 0.3970
Epoch 4/30
27/27 [==============================] - 0s 11ms/step - loss: 7537087.5000 - accuracy: 0.4630
Epoch 5/30
27/27 [==============================] - 0s 10ms/step - loss: 9022904.0000 - accuracy: 0.4502
Epoch 6/30
27/27 [==============================] - 0s 10ms/step - loss: 392054.4688 - accuracy: 0.5255
Epoch 7/30
27/27 [==============================] - 0s 9ms/step - loss: 304305.6250 - accuracy: 0.6181
Epoch 8/30
27/27 [==============================] - 0s 9ms/step - loss: 103144.3594 - accuracy: 0.6748
Epoch 9/30
27/27 [==============================] - 0s 10ms/step - loss: 148238.5312 - accuracy: 0.7002
Epoch 10/30
27/27 [==============================] - 0s 10

In [12]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

7/7 - 0s - loss: 56648256.0000 - accuracy: 0.7269

Test accuracy: 0.7268518805503845
